In [2]:
import os #Operações com o SO (arquivos)
import json #Leitura/escrita de arquivos JSON
import time #Sleep
import threading #Multithreading
import abc #Classes abstratas
from typing import Optional, Dict, List, Tuple, Any #Type hints

import tqdm #Barra de progresso
import groq #API para o Llama 3 70B
import numpy as np #Operações com arrays
import matplotlib.pyplot as plt #Plots

In [3]:
class GroqInterface:
    '''
    Interface for using the Groq API

    Implements a rate limit control for multi-threading use. 
    '''

    _client :groq.Groq = None 

    LLAMA3_70B = "llama3-70b-8192"

    inference_lock = threading.Lock()
    time_waiter_lock = threading.Lock()
    SINGLE_THREAD = True

    def __init__(self, model:Optional[str]=None, api_key:Optional[str]=None, json_mode:bool=False, system_message:Optional[str]=None, n_retry:int=5):
        '''
        GroqInterface constructor.

        Args:
            model (str, optional): model to use. Llama3 70B is used if None. Default is None
            api_key (str, optional): Groq API key to use, if None will check the environment 'GROQ_API_KEY' variable. Default is None.
            json_mode (bool): if the model need to output in JSON. Default is False.
            system_message (str): the system message to send to the model, if needed. Default is None.
            n_retyr (int): number of times to retry if the model fails (not considering RateLimitError). Default is 5.
        '''
        
        if GroqInterface._client is None:

            if api_key is None:
                api_key = os.environ.get("GROQ_API_KEY")

            if api_key is None:
                raise RuntimeError("API key is not in the environment variables ('GROQ_API_KEY' variable is not set).")

            GroqInterface._client = groq.Groq(api_key=api_key)

        if model is None:
            model = GroqInterface.LLAMA3_70B
        self._model = model

        self._system_message = system_message


        if json_mode:
            self._response_format = {"type": "json_object"}
        else:
            self._response_format = None
        self._json_mode = json_mode

        self._n_retry = n_retry

    def __call__(self, prompt:str) -> str:
        '''
        Generates the model response

        Args:
            prompt (str): prompt to send to the model.

        Returns:
            str: model response. 
        '''
        done = False
        retry_count = 0
        while not done:
            try:
                if not GroqInterface.SINGLE_THREAD:
                    GroqInterface.inference_lock.acquire()
                    GroqInterface.inference_lock.release()

                messages = []
                if self._system_message is not None:
                    messages.append({"role":"system", "content":self._system_message})
                
                messages.append({"role":"user", "content":prompt})

                chat_completion = GroqInterface._client.chat.completions.create(
                        messages=messages,
                        model=self._model,
                        response_format=self._response_format
                    )
                
                done = True
            except groq.RateLimitError as exception: #Wait
                print("ERROR")
                print(exception)
                
                GroqInterface.error = exception
                if not GroqInterface.SINGLE_THREAD:
                    if not GroqInterface.time_waiter_lock.locked():
                        GroqInterface.time_waiter_lock.acquire()
                        GroqInterface.inference_lock.acquire()
                        time.sleep(2)
                        GroqInterface.time_waiter_lock.release()
                        GroqInterface.inference_lock.release()
                else:
                    time.sleep(2)

            except KeyboardInterrupt as e: #Stop the code
                raise e
            except Exception as e: #Retry
                print("ERROR")
                print(e)
                retry_count += 1
                if retry_count >= self._n_retry:
                    raise e

        return chat_completion.choices[0].message.content

In [4]:
class Tool(abc.ABC):
    '''
    Base class for creating LLM agent tools.
    '''

    @abc.abstractmethod
    def __call__(self, query:Dict[str, str], context:str) -> Dict[str, str]:
        '''
        Execute the tool.

        Args:
            query (str): query for the tool execution.
            context (str): agent context in the tool execution moment.

        Returns:
            Dict[str, str]: tool results.
        '''
        ...

In [19]:
class MainTextGenerator(Tool, GroqInterface):


    _system_message = '''You are a informative article generator that outputs in JSON. 
The JSON object must use the schema: {'article':[{'section_name':'str', 'section_text':'str'}, {'section_name':'str', 'section_text':'str'}, ...]},

Please use a valid JSON format.'''

    _base_prompt = '''Generate a draft article for the instructions:

Theme: {theme}
Audience: {audience}
'''

    def __init__(self, model: Optional[str] = None, api_key: Optional[str] = None):
      
        super().__init__(model, api_key, True, self._system_message)

    def __call__(self, query:Dict[str, str], context:str=None) -> Dict[str, List[str]]:

        theme = query["theme"]
        audience = query["audience"]


        prompt = self._base_prompt.format(theme=theme, audience=audience)


        return json.loads(GroqInterface.__call__(self, prompt=prompt))

In [20]:
main_text_generator = MainTextGenerator()

In [21]:
query = {"theme":"prompt engineering", "audience":"general public"}

In [22]:
main_result = main_text_generator(query)

In [23]:
main_result

{'article': [{'section_name': 'Introduction',
   'section_text': 'In recent years, the field of artificial intelligence (AI) has made tremendous progress, with the development of powerful language models and other AI systems that can perform tasks that were previously thought to be the exclusive domain of humans. However, as AI systems become more advanced, the need for effective communication between humans and machines has become increasingly important. This is where prompt engineering comes in, a field that focuses on designing and optimizing the inputs or prompts that people use to interact with AI systems. In this article, we will delve into the world of prompt engineering, exploring its importance, applications, and future directions.'},
  {'section_name': 'What is Prompt Engineering?',
   'section_text': "At its core, prompt engineering is the process of designing and refining the inputs or prompts that people use to interact with AI systems. This can include natural language pr

In [28]:
class HighlightedSentencesSelector(Tool, GroqInterface):


    _system_message = '''You are an article editor for a scientific journal. Your role at this point is to select sentences to highlight.
The JSON object must use the schema: {'sentences': ['str', 'str', 'str']}, where sentences must be extracted from the article.

Please use a valid JSON format.'''

    _base_prompt = '''Select sentences to highlight. Note that few, if any, sentences should be highlighted. It must be something key to the section.

Section name: {name}
Section text: {text}
'''

    def __init__(self, model: Optional[str] = None, api_key: Optional[str] = None):
      
        super().__init__(model, api_key, True, self._system_message)

    def __call__(self, query:Dict[str, str], context:str=None) -> Dict[str, List[str]]:

        prompt = self._base_prompt.format(**query)


        return json.loads(GroqInterface.__call__(self, prompt=prompt))

In [29]:
highlight_selector = HighlightedSentencesSelector()

In [34]:
query = {"name":main_result["article"][0]["section_name"], "text":main_result["article"][0]["section_text"]}

highlight_result0 = highlight_selector(query)

In [35]:
highlight_result0

{'sentences': ['However, as AI systems become more advanced, the need for effective communication between humans and machines has become increasingly important.',
  'This is where prompt engineering comes in, a field that focuses on designing and optimizing the inputs or prompts that people use to interact with AI systems.']}

In [36]:
class DefinitionSelector(Tool, GroqInterface):


    _system_message = '''You are an article editor for a scientific journal. Your role at this point is to select terms that may be unfamiliar to the audience and create a definition to place in a text box.
The JSON object must use the schema: {'terms': [{'term':'str', 'definition':'str'}, {'term':'str', 'definition':'str'}, ...]}, where the 'term' must be extracted from the article.

Please use a valid JSON format.'''

    _base_prompt = '''Select terms to define. Note that few, if any, terms should be defined. It must be something key to the section and that the audience may not know.

Audience: {audience}
Section name: {name}
Section text: {text}
'''

    def __init__(self, model: Optional[str] = None, api_key: Optional[str] = None):
      
        super().__init__(model, api_key, True, self._system_message)

    def __call__(self, query:Dict[str, str], context:str=None) -> Dict[str, List[str]]:

        prompt = self._base_prompt.format(**query)


        return json.loads(GroqInterface.__call__(self, prompt=prompt))

In [37]:
definition_selector = DefinitionSelector()

In [38]:
query = {"name":main_result["article"][0]["section_name"], "text":main_result["article"][0]["section_text"], "audience":"general public"}

definition_result0 = definition_selector(query)

In [39]:
definition_result0

{'terms': [{'term': 'prompt engineering',
   'definition': 'The field of designing and optimizing the inputs or prompts that people use to interact with AI systems, enabling effective communication between humans and machines.'}]}

In [44]:
class ImageProposer(Tool, GroqInterface):


    _system_message = '''You are an article editor for a scientific journal. Your role at this point is to propose images to the article.
The JSON object must use the schema: {'images': [{'description':'str', 'sentence':'str'}, {'description':'str', 'sentence':'str'}, ...]}, 
where the 'description' must describe the image enough for an illustrator to create, and 'sentece' must be extracted from the article and be the sentence the image should be next to.

Please use a valid JSON format.'''

    _base_prompt = '''Propose images. Note that few, if any, images should be defined. It must be something key to the section.

Audience: {audience}
Section name: {name}
Section text: {text}
'''

    def __init__(self, model: Optional[str] = None, api_key: Optional[str] = None):
      
        super().__init__(model, api_key, True, self._system_message)

    def __call__(self, query:Dict[str, str], context:str=None) -> Dict[str, List[str]]:

        prompt = self._base_prompt.format(**query)


        return json.loads(GroqInterface.__call__(self, prompt=prompt))

In [45]:
image_proposer = ImageProposer()

In [46]:
image_result0 = image_proposer(query)

ERROR
Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{\n"images": [\n{"description": "An illustration of a human interacting with a computer, with symbols and gears in the background to represent AI systems. The human and computer should be facing each other, with a speech bubble or thought bubble coming from the human, indicating the human is communicating with the AI.", \n"sentence": "However, as AI systems become more advanced, the need for effective communication between humans and machines has become increasingly important."},\n]\n}'}}


In [47]:
image_result0

{'images': [{'description': 'An illustration showing a human interacting with a computer or robot, with speech bubbles or thought clouds representing the communication flow between the two. The human should be shown providing input to the machine, and the machine responding with output.',
   'sentence': 'However, as AI systems become more advanced, the need for effective communication between humans and machines has become increasingly important.'}]}